In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-128320")
exp = Experiment(workspace=ws, name="quick-starts-ws-128320")
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-128320
Azure region: southcentralus
Subscription id: cefbc0d9-81b3-4a98-9216-0719c151c9fe
Resource group: aml-quickstarts-128320


In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
cluster_name="cpu-cluster"
try:
    new_cluster=ComputeTarget(workspace=ws, name="cpu-cluster")
    print("Existing cluster detected, make use of it!")
except ComputeTargetException:
    print("New compute cluster creation in progress...")
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',vm_priority='dedicated',min_node=None,max_nodes=4)
    new_cluster = ComputeTarget.create(ws, "new_cluster", compute_config)    
    new_cluster.wait_for_completion(show_output=True, max_nodes=4)         
    print("Cluster is ready")                                                             

Existing cluster detected, make use of it!


In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
parameter_sampling = RandomParameterSampling( 
                    {
                        "--C":uniform(0.4,1),
                        "--max_iter":choice(0,25,50,75,100) 
                    }
) 

# Specify a Policy
early_termination_policy = BanditPolicy(evaluation_interval=1,slack_factor=0.2)
 
if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
#import packages
from azureml.core import ScriptRunConfig
from azureml.core import Environment

script_folder = './training'
os.makedirs(script_folder, exist_ok=True)
import shutil
shutil.copy('./train.py', script_folder)
script='train.py'
est=SKLearn(source_directory=script_folder,entry_script='train.py',compute_target=new_cluster)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config =  HyperDriveConfig(estimator=est,
                              hyperparameter_sampling=parameter_sampling,
                              policy=early_termination_policy,
                              primary_metric_name=PrimaryMetricName.ACCURACY,
                              primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                              max_total_runs=4,
                              max_concurrent_runs=4
                                     )

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
from azureml.train.hyperdrive.run import HyperDriveRun

hyperdrive_run = exp.submit(hyperdrive_config,show_output=True )
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_affe58ef-17d4-4437-8d52-c0c1bdd7223a
Web View: https://ml.azure.com/experiments/quick-starts-ws-128320/runs/HD_affe58ef-17d4-4437-8d52-c0c1bdd7223a?wsid=/subscriptions/cefbc0d9-81b3-4a98-9216-0719c151c9fe/resourcegroups/aml-quickstarts-128320/workspaces/quick-starts-ws-128320

Streaming azureml-logs/hyperdrive.txt

"<START>[2020-11-29T11:51:22.903857][API][INFO]Experiment created<END>\n"<START>[2020-11-29T11:51:23.6159063Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>"<START>[2020-11-29T11:51:26.093595][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2020-11-29T11:51:26.429168][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"


In [14]:
import joblib
# Get your best run.

### YOUR CODE HERE ###

best_run = hyperdrive_run.get_best_run_by_primary_metric_name()
prin("Best Run Id: ", best_run.id)

# Save the model from the run.
print(best_run.get_file_names())
hyperdrive_model = best_run.register_model(model_name = 'HyperDriveModel', model_path = 'outputs/model.joblib')

NameError: name 'hyperdrive_run' is not defined

In [9]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
src= "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds= TabularDatasetFactory.from_delimited_files(path=src)

In [10]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [13]:
#splitting the dataset
from sklearn.model_selection import train_test_split
import pandas as pd

x_train, x_test, y_train, y_test=train_test_split(x, y, train_size=0.8, test_size=0.2, random_state=42)
train_data=pd.concat((x_train,y_train),axis=1)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data='train_data',
    label_column_name='y',
    n_cross_validations=2,
    featurization='auto',
    max_concurrent_iterations=4,
    max_cores_per_iteration=4)

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
from azureml.train.automl.run import AutoMLRun

automl_run = exp.submit(automl_config, show_output=True)
RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output=True)